In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os

pd.set_option('display.max_rows', 500)

In [2]:
%%time
building = pd.read_csv('data/building_metadata.csv')
test = pd.read_csv('data/test.csv')
weather = pd.read_csv('data/weather_test.csv')

Wall time: 32 s


41697600 

In [3]:
test = test.merge(building, left_on = "building_id", right_on = "building_id", how = "left")
test = test.merge(weather, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"])

## Reduce memory

In [4]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [5]:
test = reduce_mem_usage(test)

Mem. usage decreased to 2137.11 Mb (60.3% reduction)


In [6]:
%%time
test.to_csv('test_joined.csv')

Wall time: 15min 38s


## Previous check on test data

In [ ]:
# df = pd.read_csv('data/test.csv')

In [3]:
len(df['building_id'].drop_duplicates())

1449

In [4]:
df['timestamp'].max()

'2018-12-31 23:00:00'

In [2]:
df.shape

(41697600, 4)

## Sample submission

In [3]:
df2 = pd.read_csv('sample_submission.csv')

In [4]:
df2.shape

(41697600, 2)

In [5]:
df2.head(10)

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [7]:
import numpy as np
np.sum(df2['meter_reading']>0)

0